In [1]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
pd.set_option('display.max_columns', None)

In [2]:
## get the variables

PATH_1=os.path.join('..','..','..','Weather_Data','ERA5','2010','era_2010.nc')
humidity_data = xr.open_dataset(PATH_1) ## variables
humidity_data

<xarray.Dataset> Size: 542MB
Dimensions:         (valid_time: 2920, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
    number          int64 8B ...
  * valid_time      (valid_time) datetime64[ns] 23kB 2010-01-01 ... 2010-12-3...
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
    expver          (valid_time) <U4 47kB ...
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T21:05 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
humidity_data= humidity_data.drop_vars('number') ## not sure what these two are
humidity_data= humidity_data.drop_vars('expver')
humidity_data

<xarray.Dataset> Size: 542MB
Dimensions:         (valid_time: 2920, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
  * valid_time      (valid_time) datetime64[ns] 23kB 2010-01-01 ... 2010-12-3...
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T21:05 GRIB to CDM+CF via cfgrib-0.9.1...

In [4]:
humidity_data.to_dataframe()

r         q
valid_time          pressure_level latitude longitude                     
2010-01-01 00:00:00 1000.0         49.25    -124.85    98.631653  0.004261
                                            -124.60    92.399231  0.004241
                                            -124.35    86.653137  0.004153
                                            -124.10    85.137512  0.004137
                                            -123.85    83.887512  0.004275
...                                                          ...       ...
2010-12-31 21:00:00 1000.0         24.50    -68.10     61.489914  0.008626
                                            -67.85     64.273117  0.008924
                                            -67.60     66.419601  0.009149
                                            -67.35     67.339523  0.009243
                                            -67.10     66.868820  0.009216

[67744000 rows x 2 columns]

In [5]:
## resample the data to 1 year average

year_avg = humidity_data.resample(valid_time='1YE').mean(dim='valid_time')
year_avg

<xarray.Dataset> Size: 188kB
Dimensions:         (valid_time: 1, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
  * valid_time      (valid_time) datetime64[ns] 8B 2010-12-31
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 93kB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 93kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T21:05 GRIB to CDM+CF via cfgrib-0.9.1...

In [6]:
year_avg.to_dataframe()

r         q
valid_time pressure_level latitude longitude                     
2010-12-31 1000.0         49.25    -124.85    83.282890  0.006097
                                   -124.60    80.793922  0.006133
                                   -124.35    78.617302  0.006052
                                   -124.10    76.530006  0.005966
                                   -123.85    73.657173  0.005987
...                                                 ...       ...
                          24.50    -68.10     74.635857  0.013836
                                   -67.85     74.628906  0.013831
                                   -67.60     74.626335  0.013825
                                   -67.35     74.610031  0.013815
                                   -67.10     74.607445  0.013806

[23200 rows x 2 columns]

In [7]:
year_avg = year_avg.squeeze(dim="pressure_level") ## drop the pressure level dimension
year_avg

<xarray.Dataset> Size: 188kB
Dimensions:         (valid_time: 1, latitude: 100, longitude: 232)
Coordinates:
    pressure_level  float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
  * valid_time      (valid_time) datetime64[ns] 8B 2010-12-31
Data variables:
    r               (valid_time, latitude, longitude) float32 93kB 83.28 ... ...
    q               (valid_time, latitude, longitude) float32 93kB 0.006097 ....
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T21:05 GRIB to CDM+CF via cfgrib-0.9.1...

In [8]:
SHAPE_PATH=os.path.join('..','..','..','Shapefiles','county_shapefiles','2010_county_shapefile','gz_2010_us_050_00_500k.shp')
county_gdf = gpd.read_file(SHAPE_PATH)
county_gdf

,GEO_ID,STATE,COUNTY,NAME,LSAD,CENSUSAREA,geometry
0,0500000US01029,01,029,Cleburne,County,560.100,"POLYGON ((-85.38872 33.91304, -85.38088 33.873..."
1,0500000US01031,01,031,Coffee,County,678.972,"POLYGON ((-86.03044 31.61894, -86.00408 31.619..."
2,0500000US01037,01,037,Coosa,County,650.926,"POLYGON ((-86.00928 33.10164, -86.00917 33.090..."
3,0500000US01039,01,039,Covington,County,1030.456,"POLYGON ((-86.34851 30.99434, -86.35023 30.994..."
4,0500000US01041,01,041,Crenshaw,County,608.840,"POLYGON ((-86.14699 31.68045, -86.14711 31.663..."
...,...,...,...,...,...,...,...
3216,0500000US72131,72,131,San Sebastián,Muno,70.423,"POLYGON ((-66.90748 18.25314, -66.90739 18.253..."
3217,0500000US72133,72,133,Santa Isabel,Muno,34.023,"POLYGON ((-66.37968 17.94398, -66.38029 17.943..."
3218,0500000US72137,72,137,Toa Baja,Muno,23.241,"MULTIPOLYGON (((-66.13957 18.46232, -66.13956 ..."
3219,0500000US72139,72,139,Trujillo Alto,Muno,20.764,"POLYGON ((-66.02917 18.37590, -66.02828 18.376..."


In [9]:
geometry_column=county_gdf['geometry']
geometry_column

0       POLYGON ((-85.38872 33.91304, -85.38088 33.873...
1       POLYGON ((-86.03044 31.61894, -86.00408 31.619...
2       POLYGON ((-86.00928 33.10164, -86.00917 33.090...
3       POLYGON ((-86.34851 30.99434, -86.35023 30.994...
4       POLYGON ((-86.14699 31.68045, -86.14711 31.663...
                              ...                        
3216    POLYGON ((-66.90748 18.25314, -66.90739 18.253...
3217    POLYGON ((-66.37968 17.94398, -66.38029 17.943...
3218    MULTIPOLYGON (((-66.13957 18.46232, -66.13956 ...
3219    POLYGON ((-66.02917 18.37590, -66.02828 18.376...
3220    POLYGON ((-66.85229 17.95500, -66.85280 17.955...
Name: geometry, Length: 3221, dtype: geometry

In [10]:
lat_lon=geometry_column.get_coordinates(index_parts=True)
lat_lon

x          y
0    0   -85.388717  33.913044
     1   -85.380885  33.873508
     2   -85.379455  33.866291
     3   -85.377426  33.856047
     4   -85.376403  33.850656
...             ...        ...
3220 202 -66.833718  17.989763
     203 -66.835282  17.988274
     204 -66.835429  17.986323
     205 -66.836682  17.965971
     206 -66.852288  17.955004

[837022 rows x 2 columns]

In [11]:
def single_county(var):
    
    ''' Function to interpolate the values in a single county by taking a 100 or less latitude and longitude pair.  
        Args:
        --------
             var (int): The index of the county in the shapefile.

        Returns:
        --------
            county_df: Dataframe consisting the values of the variables interpolated in the county.
    ''' 
    longitude= lat_lon.loc[(var), 'x']  # get the longitude 
    extract_val=len(longitude)        ## find number of longitude that a shapfile has

    ## if number of longitude or latitude is less than 100, that many lat-lon pairs will be extracted, if not, a 100 values
    if extract_val < 100:
        extract_val=extract_val
    else:
        extract_val=100

    longitude=longitude[:extract_val]  ## extract first 100 values or values less than 100
    
    latitude= lat_lon.loc[(var), 'y']   # get the latitude values
    latitude=latitude[:extract_val]    ## extract first 100 values or values less than 100

    lat_list=latitude.tolist()
    lon_list=longitude.tolist()

    ## find the corresponding values of the variables in the finer grid

    year_avg_finer= year_avg.interp(longitude=lon_list, latitude=lat_list)
    year_avg_finer
    
    df = year_avg_finer.squeeze(dim="valid_time") ## remove the valid_time dimension, note that this will appear as a column
                                                  ## after removed

    ## get only the 100 (or less) pair of latitude and longitude from the lat and lon list from the 100,000 (or less) rows
    get_vals=[]

    for i in range(0, extract_val):
        row=df.isel(latitude=[i], longitude=[i])
        row_df=row.to_dataframe()
        get_vals.append(row_df)

    initial_df=pd.concat(get_vals)
    initial_df=initial_df.drop('valid_time', axis=1) ## remove the valid_time column 
    
    mean=initial_df.mean() ## find the average of the 100 (or less) values
    average_df = pd.DataFrame(mean).T 

    county_df= average_df.convert_dtypes() ## the dtype of the mean values will be object, so convert that to float
    county_df
    
    return county_df

In [12]:
%%time

## include all counties

df_list=[]

for i in range(0,3221): ## A total of 3221 counties. index in shape file starts from 0 and ends in 3232
    try:
        df_list.append(single_county(i))
    except:
        pass

CPU times: total: 5min 56s
Wall time: 6min 3s


In [13]:
concatenate_df=pd.concat(df_list)
final_df=concatenate_df.reset_index(drop=True)
final_df = final_df.drop('pressure_level', axis=1)
final_df

,r,q
0,65.709291,0.008971
1,64.087602,0.009776
2,65.401402,0.009474
3,63.722471,0.009824
4,63.687011,0.009667
...,...,...
3216,<NA>,<NA>
3217,<NA>,<NA>
3218,<NA>,<NA>
3219,<NA>,<NA>


In [15]:
## lets first create a fips column in the original shape file

## lets create the fips column in the dataframe which is the sum of state and county both as dtype string.
## Note: not a numerical sum
## This step is necessary because there can be several counties with idential names. 

county_gdf['fips']=county_gdf['STATE'] + county_gdf['COUNTY']
county_gdf.dtypes
## convert the data type
county_gdf['fips']=county_gdf['fips'].astype(str).astype(int)
county_gdf.dtypes

GEO_ID          object
STATE           object
COUNTY          object
NAME            object
LSAD            object
CENSUSAREA     float64
geometry      geometry
fips             int32
dtype: object

In [17]:
county_gdf = county_gdf.drop(['GEO_ID','STATE','LSAD','COUNTY','LSAD','CENSUSAREA'], axis=1)
county_gdf

,NAME,geometry,fips
0,Cleburne,"POLYGON ((-85.38872 33.91304, -85.38088 33.873...",1029
1,Coffee,"POLYGON ((-86.03044 31.61894, -86.00408 31.619...",1031
2,Coosa,"POLYGON ((-86.00928 33.10164, -86.00917 33.090...",1037
3,Covington,"POLYGON ((-86.34851 30.99434, -86.35023 30.994...",1039
4,Crenshaw,"POLYGON ((-86.14699 31.68045, -86.14711 31.663...",1041
...,...,...,...
3216,San Sebastián,"POLYGON ((-66.90748 18.25314, -66.90739 18.253...",72131
3217,Santa Isabel,"POLYGON ((-66.37968 17.94398, -66.38029 17.943...",72133
3218,Toa Baja,"MULTIPOLYGON (((-66.13957 18.46232, -66.13956 ...",72137
3219,Trujillo Alto,"POLYGON ((-66.02917 18.37590, -66.02828 18.376...",72139


In [18]:
merged_df=pd.merge(county_gdf, final_df, left_index=True, right_index=True)
merged_df

,NAME,geometry,fips,r,q
0,Cleburne,"POLYGON ((-85.38872 33.91304, -85.38088 33.873...",1029,65.709291,0.008971
1,Coffee,"POLYGON ((-86.03044 31.61894, -86.00408 31.619...",1031,64.087602,0.009776
2,Coosa,"POLYGON ((-86.00928 33.10164, -86.00917 33.090...",1037,65.401402,0.009474
3,Covington,"POLYGON ((-86.34851 30.99434, -86.35023 30.994...",1039,63.722471,0.009824
4,Crenshaw,"POLYGON ((-86.14699 31.68045, -86.14711 31.663...",1041,63.687011,0.009667
...,...,...,...,...,...
3216,San Sebastián,"POLYGON ((-66.90748 18.25314, -66.90739 18.253...",72131,<NA>,<NA>
3217,Santa Isabel,"POLYGON ((-66.37968 17.94398, -66.38029 17.943...",72133,<NA>,<NA>
3218,Toa Baja,"MULTIPOLYGON (((-66.13957 18.46232, -66.13956 ...",72137,<NA>,<NA>
3219,Trujillo Alto,"POLYGON ((-66.02917 18.37590, -66.02828 18.376...",72139,<NA>,<NA>


In [19]:
county_df=merged_df.dropna()
county_df

,NAME,geometry,fips,r,q
0,Cleburne,"POLYGON ((-85.38872 33.91304, -85.38088 33.873...",1029,65.709291,0.008971
1,Coffee,"POLYGON ((-86.03044 31.61894, -86.00408 31.619...",1031,64.087602,0.009776
2,Coosa,"POLYGON ((-86.00928 33.10164, -86.00917 33.090...",1037,65.401402,0.009474
3,Covington,"POLYGON ((-86.34851 30.99434, -86.35023 30.994...",1039,63.722471,0.009824
4,Crenshaw,"POLYGON ((-86.14699 31.68045, -86.14711 31.663...",1041,63.687011,0.009667
...,...,...,...,...,...
3138,Niobrara,"POLYGON ((-104.05298 42.85955, -104.05286 42.7...",56027,55.440386,0.004788
3139,Platte,"POLYGON ((-104.77417 42.60996, -104.76422 42.6...",56031,53.748086,0.004545
3140,Sweetwater,"POLYGON ((-109.05008 41.00066, -109.17368 41.0...",56037,58.363303,0.003648
3141,Washakie,"POLYGON ((-107.12892 43.99455, -107.12797 43.9...",56043,53.062419,0.003824


In [20]:
type(county_df)

geopandas.geodataframe.GeoDataFrame

In [21]:
## convert into pandas dataframe without the geometry column
pd.options.display.float_format = '{:.15f}'.format ## see 15 decimal places of the numbers
county_var=pd.DataFrame(county_df.drop(columns='geometry')) 
county_var

,NAME,fips,r,q
0,Cleburne,1029,65.709290953976236,0.008971238939755
1,Coffee,1031,64.087601555155445,0.009776382310924
2,Coosa,1037,65.401402257280296,0.009474377114962
3,Covington,1039,63.722471195196370,0.009823778143466
4,Crenshaw,1041,63.687011214291942,0.009667189015053
...,...,...,...,...
3138,Niobrara,56027,55.440385821295664,0.004788146983723
3139,Platte,56031,53.748086432814276,0.004544928871319
3140,Sweetwater,56037,58.363303158239681,0.003648367785138
3141,Washakie,56043,53.062418947239131,0.003823781620200


In [22]:
county_var = county_var.sort_values(by='fips')
county_var

,NAME,fips,r,q
14,Autauga,1001,62.689693684141481,0.009403329795795
15,Baldwin,1003,68.481218149674973,0.010900847042377
16,Barbour,1005,64.462901234293724,0.009733833025909
17,Bibb,1007,64.103444382726352,0.009355870900120
18,Blount,1009,65.862652924185539,0.008997889824490
...,...,...,...,...
3140,Sweetwater,56037,58.363303158239681,0.003648367785138
3133,Teton,56039,67.463731317950760,0.003624777878190
3134,Uinta,56041,59.520474922851420,0.003591308332443
3141,Washakie,56043,53.062418947239131,0.003823781620200


In [23]:
county_var.to_pickle('era_2010.pkl')